In [1]:
import numpy as np
import pandas as pd
import pickle

In [2]:
train = pd.read_csv('./raws/train_data.csv')
test = pd.read_csv('./raws/test_data.csv')

In [3]:
def featurecreation(df):
    df = df.loc[:, 'lat':'sst_diff']
    df.drop(columns=['year', 'month', 'YMD'], inplace=True)

    df['beach_rate'] = df['beach_length'] / df['coast_length']
    df['aicial_rate'] = df['aicial_length'] / df['coast_length']

    df['YMDstyle'] = [str(str(i).count('-')) +'_'+ str(str(i).count('.')) for i in list(df['YMD'])]
    df['YMDstyle'].replace(['0_0', '0_1', '0_2', '0_4', '1_2', '1_4', '2_12'], [2, 1, 3, np.nan, np.nan, 4, np.nan])

    return df

In [4]:
train_ = featurecreation(train)
test_ = featurecreation(test)

In [5]:
pickle.dump(train_, open('./processed/lgb_train.pkl', 'wb'))
pickle.dump(test_, open('./processed/lgb_test.pkl', 'wb'))

In [6]:
test_

,lat,lon,area,depth_original,cliff_length,aicial_length,beach_length,coast_length,coastal_dist,cold_sst,...,hist_warm_sst,river_area,river_dist,Salinity_annual,warm_sst,sst_annual,sst_ymd,sst_diff,beach_rate,aicial_rate
0,24.352222,124.202674,20.0,0.5,0.000000,725.621704,306.701050,1032.322754,280.0,23.076576,...,31.449434,2986.0,0.618851,34.553608,29.288065,26.149386,25.052774,1.096612,0.297098,0.702902
1,24.420028,124.078583,20.0,NaN,0.000000,0.000000,1421.016357,1421.016357,225.0,23.083557,...,31.305880,1893.0,6.964348,34.548206,29.380964,26.148504,29.674129,-3.525625,1.000000,0.000000
2,26.494197,127.836013,100.0,13.8,73.183418,80.595177,984.422791,1138.201416,100.0,21.049788,...,31.274185,887.0,7.078082,34.674667,29.131052,24.911836,21.202750,3.709086,0.864893,0.070809
3,26.183639,127.319361,20.0,NaN,0.000000,0.000000,29.648212,29.648212,60.0,21.556902,...,31.139191,3284.0,34.868923,34.673969,28.939396,25.176858,29.066452,-3.889594,1.000000,0.000000
4,24.761089,125.269009,20.0,-0.5,0.000000,34.796036,372.873108,407.669128,725.0,22.642460,...,31.021915,2986.0,116.736473,34.572544,29.216471,25.902807,26.865000,-0.962193,0.914646,0.085354
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4034,26.702075,127.873347,20.0,0.4,0.000000,20.594961,581.052856,601.647827,380.0,20.995081,...,31.134930,1126.0,5.700911,34.673130,28.988804,24.816042,28.048267,-3.232224,0.965769,0.034231
4035,24.337028,124.232694,20.0,NaN,0.000000,0.000000,64.563194,64.563194,860.0,23.096041,...,31.475069,2986.0,3.125924,34.554840,29.274986,26.163280,25.444267,0.719013,1.000000,0.000000
4036,26.493121,127.839918,100.0,0.1,76.439651,133.127228,1403.852905,1613.419800,70.0,21.044603,...,31.271669,887.0,7.141524,34.672771,29.135160,24.907287,21.192750,3.714537,0.870110,0.082512
4037,24.068778,123.804194,20.0,NaN,73.660492,0.000000,685.145386,758.805908,110.0,23.469366,...,31.601223,2896.0,24.530298,34.514717,29.342030,26.421852,23.431500,2.990352,0.902926,0.000000
